In [1]:
import pandas as pd

# List of your processed file paths
file_paths = [
    r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Processed_Data\Bangalore_preprocessed.csv",
    r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Processed_Data\Bangalore_preprocessed.csv",
    r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Processed_Data\Delhi_preprocessed.csv",
    r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Processed_Data\Hyderabad_preprocessed.csv",
    r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Processed_Data\Jaipur_preprocessed.csv",
    r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Processed_Data\Kolkata_preprocessed.csv"
]

# Initialize an empty list to hold the DataFrames
dataframes = []

# Loop through each file and read it into a DataFrame
for file in file_paths:
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate all DataFrames into one
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv("merged_car.csv", index=False)


In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from scipy import stats

# Load the dataset
df = pd.read_csv(r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Merged_car\merged_car.csv")

# 1. Handling Missing Values

# For numerical columns, use mean imputation
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())

# For categorical columns, use mode imputation or create a new category for missing values
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].mode()[0])  # Updated to avoid FutureWarning

# 2. Standardising Data Formats

# Example: Removing units and commas from 'kms_driven'
if 'kms_driven' in df.columns:
    df['kms_driven'] = df['kms_driven'].str.replace(' Kms', '', regex=False)  # Remove ' Kms'
    df['kms_driven'] = df['kms_driven'].str.replace(',', '', regex=False)   # Remove commas
    df['kms_driven'] = pd.to_numeric(df['kms_driven'], errors='coerce')     # Convert to numeric, coerce errors to NaN

# Ensure all columns are of the correct data type
print("Data Types of Columns:")
print(df.dtypes)

# 3. Encoding Categorical Variables

# Print column names to verify
print("Categorical Columns: ", categorical_cols)

# One-Hot Encoding (for nominal categorical variables)
# Identify actual nominal columns and update the list
nominal_columns = [col for col in categorical_cols if df[col].nunique() < 10]  # Example criterion
df = pd.get_dummies(df, columns=nominal_columns, drop_first=True)

# Label Encoding (for ordinal categorical variables)
# Identify actual ordinal columns and update the list
ordinal_columns = []  # Replace with actual ordinal column names if any
for col in ordinal_columns:
    if col in df.columns:
        label_encoder = LabelEncoder()
        df[col] = label_encoder.fit_transform(df[col])

# 4. Normalizing Numerical Features

# Choose between Min-Max Scaling or Standard Scaling
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Ensure no NaN values before scaling
df[numerical_cols] = df[numerical_cols].fillna(0)  # Fill NaNs with 0 or another appropriate value

# Min-Max Scaling
min_max_scaler = MinMaxScaler()
df[numerical_cols] = min_max_scaler.fit_transform(df[numerical_cols])

# Alternatively, you can use Standard Scaling
# standard_scaler = StandardScaler()
# df[numerical_cols] = standard_scaler.fit_transform(df[numerical_cols])

# 5. Removing Outliers

# Using IQR Method
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df[col] >= (Q1 - 1.5 * IQR)) & (df[col] <= (Q3 + 1.5 * IQR))]

# Alternatively, use Z-Score Method
# df = df[(np.abs(stats.zscore(df[numerical_cols])) < 3).all(axis=1)]

# Save the cleaned dataset
df.to_csv('cleaned_merged_car.csv', index=False)

print("Data preprocessing completed successfully.")


Data Types of Columns:
City                       object
car_links                  object
fuel_type                  object
body_type                  object
kilometers_driven           int64
transmission               object
owner                      object
oem                        object
model                      object
year                        int64
variant                    object
price                      object
registration_year         float64
insurance_validity         object
fuel_type_overview         object
seats                     float64
kms_driven                  int64
rto                        object
comfort_convenience        object
interior_features          object
exterior_features          object
safety_features            object
entertainment_features     object
mileage                   float64
engine                    float64
max_power                  object
torque                    float64
wheel_size                float64
bhp                      